In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy import stats

In [6]:
# Load the lora_image_count.pkl file
with open('lora_image_count.pkl', 'rb') as f:
    lora_image_count = pickle.load(f)
